In [1]:
import math
from collections import Counter
from tqdm import tqdm
import tensorflow as tf
from transe import build_model
from utils import load_data, data_generate

In [2]:
train_file = 'WN18/train.txt'
valid_file = 'WN18/valid.txt'
test_file = 'WN18/train.txt'

train = load_data(train_file)
valid = load_data(valid_file)
test = load_data(test_file)

# words实际上是实体和关系的字符串
words = Counter()
for x in train:
    words.update(x)
for x in valid:
    words.update(x)
for x in test:
    words.update(x)
print(len(words))
# 大约是words数量的10倍
n_ent_rel = 50 * 10000

batch_size = 1024 * 4
n_batch_train = math.ceil(len(train) / batch_size)
n_batch_valid = math.ceil(len(valid) / batch_size)

40961


In [3]:
model, e2i, emb = build_model(n_ent_rel)

model.fit_generator(
    data_generate(train, batch_size),
    steps_per_epoch=n_batch_train,
    epochs=100,
    validation_data=data_generate(valid, batch_size),
    validation_steps=n_batch_valid,
    callbacks=[
        tf.keras.callbacks.TensorBoard('./logs')
    ]
)

Epoch 1/100
35/35 [==============================] - 17s 475ms/step - loss: 0.9870 - val_loss: 0.9787
Epoch 2/100
35/35 [==============================] - 15s 426ms/step - loss: 0.9429 - val_loss: 0.9532
Epoch 3/100
35/35 [==============================] - 15s 436ms/step - loss: 0.8983 - val_loss: 0.9226
Epoch 4/100
35/35 [==============================] - 15s 433ms/step - loss: 0.8511 - val_loss: 0.8863
Epoch 5/100
35/35 [==============================] - 15s 433ms/step - loss: 0.8009 - val_loss: 0.8493
Epoch 6/100
35/35 [==============================] - 15s 440ms/step - loss: 0.7503 - val_loss: 0.8133
Epoch 7/100
35/35 [==============================] - 15s 437ms/step - loss: 0.7032 - val_loss: 0.7778
Epoch 8/100
35/35 [==============================] - 15s 439ms/step - loss: 0.6611 - val_loss: 0.7506
Epoch 9/100
35/35 [==============================] - 15s 431ms/step - loss: 0.6200 - val_loss: 0.7174
Epoch 10/100
35/35 [==============================] - 15s 429ms/step - loss: 0.579

Epoch 80/100
35/35 [==============================] - 17s 494ms/step - loss: 0.0203 - val_loss: 0.1157
Epoch 81/100
35/35 [==============================] - 17s 481ms/step - loss: 0.0191 - val_loss: 0.1127
Epoch 82/100
35/35 [==============================] - 17s 499ms/step - loss: 0.0190 - val_loss: 0.1123
Epoch 83/100
35/35 [==============================] - 18s 506ms/step - loss: 0.0186 - val_loss: 0.1099
Epoch 84/100
35/35 [==============================] - 17s 478ms/step - loss: 0.0175 - val_loss: 0.1079
Epoch 85/100
35/35 [==============================] - 16s 463ms/step - loss: 0.0170 - val_loss: 0.1062
Epoch 86/100
35/35 [==============================] - 17s 472ms/step - loss: 0.0169 - val_loss: 0.1048
Epoch 87/100
35/35 [==============================] - 17s 494ms/step - loss: 0.0165 - val_loss: 0.1052
Epoch 88/100
35/35 [==============================] - 17s 500ms/step - loss: 0.0159 - val_loss: 0.1031
Epoch 89/100
35/35 [==============================] - 18s 505ms/step - lo

In [4]:
# Save model
e2i.save('/tmp/e2i')
emb.save('/tmp/emb')

INFO:tensorflow:Assets written to: /tmp/e2i/assets
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/emb/assets


以下是测试部分，用来计算测试数据集的HIT@10

In [3]:
entities = sorted(words.keys())
rel = [x for x in entities if x.startswith('_')]
entities = [x for x in entities if not x.startswith('_')]
entity_ind = {k: v for v, k in enumerate(entities)}

In [4]:
e2i = tf.keras.models.load_model('/tmp/e2i')
emb = tf.keras.models.load_model('/tmp/emb')

In [5]:
print(len(rel), len(entities))

18 40943


In [6]:
ent_vecs = emb(e2i(tf.constant([entities]))).numpy()[0]
rel_vecs = emb(e2i(tf.constant([rel]))).numpy()[0]

In [7]:
print(ent_vecs.shape, rel_vecs.shape)

(40943, 64) (18, 64)


In [8]:
entities_vecs = emb(e2i(tf.constant([[x] for x in entities])))[:, 0]
test_head = emb(e2i(tf.constant([[x[0]] for x in test])))[:, 0]
test_rel = emb(e2i(tf.constant([[x[1]] for x in test])))[:, 0]
pred_test_tail = test_head + test_rel

In [9]:
good, bad = 0, 0
pbar = tqdm(enumerate(test), total=len(test))
for i, (_, _, tail) in pbar:
    tail_ind = entity_ind[tail]
    vec = pred_test_tail[i]
    _, dis = tf.linalg.normalize(vec - entities_vecs, axis=1)
    ind = tf.argsort(tf.squeeze(dis)).numpy()[:10]
    sign = tail_ind in ind
    if sign:
        good += 1
    else:
        bad += 1
    pbar.set_description(
        f'good: {good}, bad: {bad}, total: {good + bad}, hit@10: {good / (good + bad):.4f}'
    )

good: 119930, bad: 21512, total: 141442, hit@10: 0.8479: 100%|██████████| 141442/141442 [05:52<00:00, 401.04it/s]


In [10]:
# https://github.com/thunlp/KB2E
# Hit@10(raw) transE = 75.4 or 78.9
print(good, bad, len(test), good / len(test))

119930 21512 141442 0.8479093904215156
